In [1]:
import os
import json
import asyncio
import websockets
import requests
from alpaca_trade_api.rest import REST, TimeFrame
from dotenv import load_dotenv

In [2]:
load_dotenv()
APCA_API_KEY_ID = os.getenv('APCA_API_KEY_ID')
APCA_API_SECRET_KEY = os.getenv('APCA_API_SECRET_KEY')

In [3]:
alpaca = REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, base_url="https://paper-api.alpaca.markets")

In [4]:
ws_url = "wss://stream.data.alpaca.markets/v1beta1/news"

In [ ]:
async def authenticate_and_subscribe(ws):
    # Authenticate with Alpaca's WebSocket
    auth_msg = {
        "action": "auth",
        "key": APCA_API_KEY_ID,
        "secret": APCA_API_SECRET_KEY
    }
    await ws.send(json.dumps(auth_msg))

    # Subscribe to all news feeds
    subscribe_msg = {
        "action": "subscribe",
        "news": ["*"]  # ["TSLA"] for specific stocks
    }
    await ws.send(json.dumps(subscribe_msg))